## Detect Offshore wind Turbine from Sentinel-1 SAR image

In [1]:
import requests
import json
import numpy as np
from sklearn.cluster import DBSCAN, KMeans
from sklearn import metrics
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import geopandas as gpd
import pandas as pd
#from template file
from pyproj import Proj, Transformer
import os
import rasterio
import rasterio.features
from rasterio.windows import from_bounds, Window
import pystac_client
import planetary_computer as pc
import stackstac
import rtree
import fiona
from shapely.geometry import mapping
import shapely.geometry
from shapely.geometry import Point, box, Polygon
from IPython.display import Image
import shutil
from rasterio.enums import Resampling
from affine import Affine

In [2]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_wind = """
[out:json];
(
  node["generator:source"="wind"]["offshore"="yes"];
);
out body;
"""

overpass_oil = """
[out:json];
(
  node["man_made"="offshore_platform"];
);
out body;
"""

response1 = requests.get(overpass_url, 
                        params={'data': overpass_wind})

response2 = requests.get(overpass_url, 
                        params={'data': overpass_oil})

wind_data = response1.json()
oil_data = response2.json()
print("First 2 line of data",json.dumps(wind_data['elements'][:2], indent=4))
print("First 2 line of data",json.dumps(oil_data['elements'][:2], indent=4))

First 2 line of data [
    {
        "type": "node",
        "id": 25365238,
        "lat": 55.706953,
        "lon": 12.6683513,
        "tags": {
            "generator:method": "wind_turbine",
            "generator:output:electricity": "2000 kW",
            "generator:source": "wind",
            "generator:type": "horizontal_axis",
            "height": "102",
            "offshore": "yes",
            "operator": "HOFOR",
            "power": "generator",
            "ref": "M1",
            "seamark:landmark:category": "windmotor",
            "seamark:light:1:category": "air_obstruction",
            "seamark:light:1:character": "F",
            "seamark:light:1:colour": "red",
            "seamark:light:2:character": "Mo",
            "seamark:light:2:colour": "yellow",
            "seamark:light:2:group": "U",
            "seamark:light:2:period": "15",
            "seamark:light:2:range": "5",
            "seamark:type": "landmark",
            "source": "bing"
        }
  

In [3]:
oilrig_data = oil_data['elements']
windmill_data = wind_data['elements']
print("Total numbers of wind hill: ", len(windmill_data), "Oil data: ", len(oilrig_data))


Total numbers of wind hill:  5193 Oil data:  3204


In [4]:
# Windmill data
windmill_points = [(1, element['lat'], element['lon']) for element in windmill_data if 'lat' in element and 'lon' in element]

# Oilrig data
oilrig_points = [(2, element['lat'], element['lon']) for element in oilrig_data if 'lat' in element and 'lon' in element]

# Combine oilrig and windmill points with label
combined_points = windmill_points + oilrig_points

# Convert combined_points to a DataFrame
df = pd.DataFrame(combined_points, columns=['label', 'latitude', 'longitude'])

print(df.head())

   label   latitude  longitude
0      1  55.706953  12.668351
1      1  55.705335  12.668828
2      1  55.703706  12.669273
3      1  55.702079  12.669646
4      1  55.700449  12.669966


In [5]:
df['geometry'] = df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
print(gdf.head())

   label   latitude  longitude                   geometry
0      1  55.706953  12.668351  POINT (12.66835 55.70695)
1      1  55.705335  12.668828  POINT (12.66883 55.70534)
2      1  55.703706  12.669273  POINT (12.66927 55.70371)
3      1  55.702079  12.669646  POINT (12.66965 55.70208)
4      1  55.700449  12.669966  POINT (12.66997 55.70045)


In [6]:
# Create bounding boxes
kms_per_radian = 6371.0088
epsilon = 2 / kms_per_radian  # 2 kilometers is the maximum distance
db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(gdf[['latitude', 'longitude']].values))
gdf['cluster'] = db.labels_

gdf.groupby('cluster')['geometry']

print(gdf)

      label   latitude  longitude                    geometry  cluster
0         1  55.706953  12.668351   POINT (12.66835 55.70695)        0
1         1  55.705335  12.668828   POINT (12.66883 55.70534)        0
2         1  55.703706  12.669273   POINT (12.66927 55.70371)        0
3         1  55.702079  12.669646   POINT (12.66965 55.70208)        0
4         1  55.700449  12.669966   POINT (12.66997 55.70045)        0
...     ...        ...        ...                         ...      ...
8392      2  28.550691 -91.000287  POINT (-91.00029 28.55069)     1584
8393      2  28.778744 -90.748014  POINT (-90.74801 28.77874)     1036
8394      2  40.799252  24.496952   POINT (24.49695 40.79925)     1585
8395      2  40.798592  24.498239   POINT (24.49824 40.79859)     1585
8396      2  40.798491  24.497132   POINT (24.49713 40.79849)     1585

[8397 rows x 5 columns]


In [7]:
# Compute bounding box for each cluster
def compute_bbox(points):
    minx, miny, maxx, maxy = points.geometry.total_bounds
    return box(minx, miny, maxx, maxy)

bboxes = gdf.groupby('cluster')['geometry'].apply(compute_bbox).reset_index()

bboxes[['minx', 'miny', 'maxx', 'maxy']] = bboxes['geometry'].bounds

gdf3 = gdf.merge(bboxes[['cluster', 'minx', 'miny', 'maxx', 'maxy']], on='cluster')


In [8]:
gdf3

,label,latitude,longitude,geometry,cluster,minx,miny,maxx,maxy
0,1,55.706953,12.668351,POINT (12.66835 55.70695),0,12.668351,55.675894,12.670758,55.706953
1,1,55.705335,12.668828,POINT (12.66883 55.70534),0,12.668351,55.675894,12.670758,55.706953
2,1,55.703706,12.669273,POINT (12.66927 55.70371),0,12.668351,55.675894,12.670758,55.706953
3,1,55.702079,12.669646,POINT (12.66965 55.70208),0,12.668351,55.675894,12.670758,55.706953
4,1,55.700449,12.669966,POINT (12.66997 55.70045),0,12.668351,55.675894,12.670758,55.706953
...,...,...,...,...,...,...,...,...,...
8392,2,29.607388,-93.216028,POINT (-93.21603 29.60739),1583,-93.216028,29.607388,-93.216028,29.607388
8393,2,28.550691,-91.000287,POINT (-91.00029 28.55069),1584,-91.000287,28.550691,-91.000287,28.550691
8394,2,40.799252,24.496952,POINT (24.49695 40.79925),1585,24.496952,40.798491,24.498239,40.799252
8395,2,40.798592,24.498239,POINT (24.49824 40.79859),1585,24.496952,40.798491,24.498239,40.799252


gdf['latitude', 'longitude'] --- Point
bboxes.bounds --- bbox
gdf['label']  --- label

In [9]:
ep = 1 #deg
combined_values = []
for minx, miny, maxx, maxy in zip(gdf3['minx'], gdf3['miny'], gdf3['maxx'], gdf3['maxy']):
    combined_values.append([minx-ep, miny-ep, maxx+ep, maxy+ep])

bbox_gdf2 = gpd.GeoDataFrame({'geometry': combined_values, 'cluster': gdf3['cluster'], 'label': gdf3['label']})
bbox_gdf2 = bbox_gdf2.drop_duplicates(subset='cluster')

bbox_gdf2

,geometry,cluster,label
0,"[11.6683513, 54.675894, 13.670758, 56.706953]",0,1
20,"[0.4103527, 50.5749548, 2.5714078000000002, 52...",1,1
195,"[3.3667778000000004, 51.579553, 5.4598889, 53....",2,1
231,"[3.1886132, 51.5715658, 5.2601502, 53.6053721]",3,1
292,"[5.4155719, 52.9164217, 7.6233333, 55.089183]",4,1
...,...,...,...
8390,"[-93.0701602, 27.6497731, -91.0701602, 29.6497...",1581,2
8391,"[-94.4009372, 27.9171295, -92.4009372, 29.9171...",1582,2
8392,"[-94.216028, 28.6073881, -92.216028, 30.6073881]",1583,2
8393,"[-92.0002868, 27.5506909, -90.0002868, 29.5506...",1584,2


# Approach 1 Using 256 * 256 image
- First we use the cluster to get the rough image to download. 

In [10]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1"
)

inProj = 'epsg:4326'
ml_dataset = []  # the machine learning dataset
ml_labels = []  # the labels, which will be bounding boxes in pixel coordinates

NumberofLoop = bbox_gdf2['cluster']

window_size = 256
# fig, axs = plt.subplots(5, 5, figsize=(10, 10))

In [11]:
def extract_window_around_nearest_pixel(data_slice, point, transformer, window_size):
    x_geo, y_geo = transformer.transform(point['latitude'], point['longitude'])
    
    # Nearest pixel to the point
    nearest_pixel = data_slice.sel(x=x_geo, y=y_geo, method='nearest')
    
    # Indexes of the nearest pixel
    nearest_x_index = (data_slice.x==nearest_pixel.x).argmax().item()
    nearest_y_index = (data_slice.y==nearest_pixel.y).argmax().item()

    # Index ranges for window
    x_start = max(0, nearest_x_index - window_size//2)
    x_end = min(data_slice.shape[1], nearest_x_index + window_size//2)
    y_start = max(0, nearest_y_index - window_size//2)
    y_end = min(data_slice.shape[0], nearest_y_index + window_size//2)
    # print(x_start,x_end, y_start, y_end)
    # Extract the window around the pixel
    window = data_slice[y_start:y_end, x_start:x_end].values
    return window

In [12]:
#f"OilRig/Label{row2['label']}_ClusterNo{row2['cluster']}_ImageName_{str(ds[i].id.values)}.png"
gdf3

,label,latitude,longitude,geometry,cluster,minx,miny,maxx,maxy
0,1,55.706953,12.668351,POINT (12.66835 55.70695),0,12.668351,55.675894,12.670758,55.706953
1,1,55.705335,12.668828,POINT (12.66883 55.70534),0,12.668351,55.675894,12.670758,55.706953
2,1,55.703706,12.669273,POINT (12.66927 55.70371),0,12.668351,55.675894,12.670758,55.706953
3,1,55.702079,12.669646,POINT (12.66965 55.70208),0,12.668351,55.675894,12.670758,55.706953
4,1,55.700449,12.669966,POINT (12.66997 55.70045),0,12.668351,55.675894,12.670758,55.706953
...,...,...,...,...,...,...,...,...,...
8392,2,29.607388,-93.216028,POINT (-93.21603 29.60739),1583,-93.216028,29.607388,-93.216028,29.607388
8393,2,28.550691,-91.000287,POINT (-91.00029 28.55069),1584,-91.000287,28.550691,-91.000287,28.550691
8394,2,40.799252,24.496952,POINT (24.49695 40.79925),1585,24.496952,40.798491,24.498239,40.799252
8395,2,40.798592,24.498239,POINT (24.49824 40.79859),1585,24.496952,40.798491,24.498239,40.799252


In [13]:
root_dir = "Data256"
class_dirs = ["Widmill_Filtered", "OilRig_Filtered"]

In [14]:
for index, row in bbox_gdf2.iterrows():
    # Access data
    bbox = row['geometry']
    cluster = row['cluster']
    label = row['label']
    # Print or manipulate data
    search = catalog.search(
        collections=["sentinel-1-rtc"],
        bbox=bbox,
        max_items=5
    )
    items = search.item_collection()
    item = items[0] # an assumption that most of image that retrive are having similar coordinate system or very similar. This cause potential discrepancy.
    outProj = f'epsg:{item.properties["proj:epsg"]}'
    #generate tranformation based on the image
    transformer = Transformer.from_crs(inProj, outProj)
    ds = stackstac.stack(pc.sign(items), bounds_latlon=bbox, epsg=item.properties['proj:epsg'], resolution=10)
    PointToDownload = gdf3[gdf3['cluster'] == cluster]
    
    for i in range(ds.shape[0]): # Loop over the 'time' dimension
        data_slice = ds[i, 0]
        for i2, row2 in PointToDownload.iterrows():
            window = extract_window_around_nearest_pixel(data_slice, row2, transformer, window_size)
            # Index ranges for window
            if np.isnan(window).mean() < 0.2 and window.shape == (window_size, window_size) and np.nanmax(window) > 0.1:
                img = np.nan_to_num(window, nan=0.00000001)
                if row2['label'] == 1:
                    colsr = plt.imshow(img, vmax=0.1)
                    print(f"Point Number: {i2}")
                    plt.axis('off')
                    plt.savefig(os.path.join(root_dir, f"{class_dirs[0]}/Label{row2['label']}_Point{i2}_ClusterNo{row2['cluster']}_ImageName_{str(ds[i].id.values)}.png"), bbox_inches='tight', pad_inches=0)
                    plt.close()
                elif row2['label'] == 2:
                    colsr = plt.imshow(img, vmax=0.1)
                    print(f"Point Number: {i2}")
                    plt.axis('off')
                    plt.savefig(os.path.join(root_dir, f"{class_dirs[1]}/Label{row2['label']}_Point{i2}_ClusterNo{row2['cluster']}_ImageName_{str(ds[i].id.values)}.png"), bbox_inches='tight', pad_inches=0)
                    plt.close()

/srv/conda/envs/notebook/lib/python3.11/site-packages/stackstac/prepare.py:363: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


Point Number: 0
Point Number: 1
Point Number: 2
Point Number: 3
Point Number: 4
Point Number: 5
Point Number: 6
Point Number: 7
Point Number: 8
Point Number: 9
Point Number: 10
Point Number: 11
Point Number: 12
Point Number: 13
Point Number: 14
Point Number: 15
Point Number: 16
Point Number: 17
Point Number: 18
Point Number: 19


/srv/conda/envs/notebook/lib/python3.11/site-packages/stackstac/prepare.py:363: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


Point Number: 21
Point Number: 22
Point Number: 23
Point Number: 24
Point Number: 26
Point Number: 29
Point Number: 34
Point Number: 37
Point Number: 38
Point Number: 40
Point Number: 43
Point Number: 44
Point Number: 46
Point Number: 49
Point Number: 50
Point Number: 52
Point Number: 55
Point Number: 57
Point Number: 59
Point Number: 61
Point Number: 62
Point Number: 64
Point Number: 67
Point Number: 68
Point Number: 72
Point Number: 74
Point Number: 78
Point Number: 80
Point Number: 82
Point Number: 84
Point Number: 85
Point Number: 87
Point Number: 90
Point Number: 92
Point Number: 96
Point Number: 98
Point Number: 100
Point Number: 103
Point Number: 105
Point Number: 107
Point Number: 109
Point Number: 110
Point Number: 112
Point Number: 114
Point Number: 116
Point Number: 118
Point Number: 122
Point Number: 124
Point Number: 126
Point Number: 129
Point Number: 131
Point Number: 134
Point Number: 135
Point Number: 137
Point Number: 140
Point Number: 141
Point Number: 145
Point Numb

/srv/conda/envs/notebook/lib/python3.11/site-packages/stackstac/prepare.py:363: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


...

Point Number: 8392


/srv/conda/envs/notebook/lib/python3.11/site-packages/stackstac/prepare.py:363: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


Point Number: 8393


/srv/conda/envs/notebook/lib/python3.11/site-packages/stackstac/prepare.py:363: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


Point Number: 8394
Point Number: 8395
Point Number: 8396
